![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio C - Calibración de los SLM </font>
</div>

- **Fecha**: *
        
- **Alumno**: Alex G Recuenco 


# Introducción

En este ejercicio vamos a trabajar con la calibración del SLM para encontrar unas condiciones de modulación de amplitud y modulación de fase. Para ello, hay que diseñar una función de mérito para cada una de los dos tipos de modulación y encontrar una configuración de ángulos para cada una mediante una optimización.

## Cargar calibración del SLM

Hay que realizar estos pasos para cargar la calibración como un objeto *Jones_vector* de *pypol*.

In [ ]:
import numpy as np
from py_pol.jones_matrix import Jones_matrix
from py_pol.jones_vector import Jones_vector

In [ ]:
from collections import namedtuple


AnglesElements = namedtuple("AnglesElements", ["pg", "qg", "qa", "pa"])
AnglesField = namedtuple("AnglesField", ["phi_a", "xi_a", "phi_g", "xi_g"])


def field_to_elements(angles: AnglesField) -> AnglesElements:
    assert isinstance(angles, AnglesField)
    phi_a, xi_a, phi_g, xi_g = angles
    qg, pg = phi_g, phi_g - xi_g
    qa, pa = phi_a, phi_a + xi_a
    return AnglesElements(pg, qg, qa, pa)


def angles_degree_to_rad(angles_360: AnglesField) -> AnglesField:
    assert isinstance(angles_360, AnglesField)
    a1, a2, a3, a4 = angles_360
    ratio = np.pi / 180
    return AnglesField(a1 * ratio, a2 * ratio, a3 * ratio, a4 * ratio)


def transmission(angles: AnglesElements, J: Jones_matrix) -> Jones_vector:
    assert isinstance(angles, AnglesElements)
    (pg, qg, qa, pa) = angles
    Ein = Jones_vector().circular_light(intensity=2)
    P_g = Jones_matrix().diattenuator_perfect(azimuth=pg)
    Q_g = Jones_matrix().quarter_waveplate(azimuth=qg)
    Q_a = Jones_matrix().quarter_waveplate(azimuth=qa)
    P_a = Jones_matrix().diattenuator_perfect(azimuth=pa)

    Eg: Jones_vector = Q_g * P_g * Ein
    Efin: Jones_vector = P_a * Q_a * J * Eg
    return Efin


def field_transmission(angles: AnglesField, J: Jones_matrix) -> Jones_vector:
    assert isinstance(angles, AnglesField)
    angles_elements = field_to_elements(angles)
    return transmission(angles_elements, J)


def parameters_intensity_phase(E: Jones_vector):
    return E.parameters.intensity(), (E.parameters.global_phase() % 2 * np.pi)


def parameters_az_el(E: Jones_vector):
    return E.parameters.azimuth_ellipticity()

In [ ]:
def load_SLM(filename):  # Cargar datos del archivo NPZ (grabado con np.savez)
    data = np.load(filename)
    # Definir una lista con las componentes de la matriz de Jones: J00, J01, J10 y J11
    components = [
        data["J0"] * np.exp(1j * 0),
        data["J1"] * np.exp(1j * (data["d1"] - data["d0"])),
        data["J2"] * np.exp(1j * (data["d2"] - data["d0"])),
        data["J3"] * np.exp(1j * (data["d3"] - data["d0"])),
    ]
    # Crear el objeto py_pol
    Jslm = Jones_matrix()
    Jslm.from_components(components)
    return Jslm


Jslm = load_SLM("SLM_Jones_components.npz")

## Sistema

El sistema será igual que en el laboratorio: un haz inicial de polarización circular a derechas, un generador de estados (PSG) formado por un polarizador perfecto y una lámina de cuarto de onda que pueden rotar, el SLM, y un analizador de estados (PSA) formado por una lámina de cuarto de onda y un polarizador perfecto que pueden rotar.

# Calibración completa

Vamos a calibrar la matriz del SLM que tenemos (como si no supiéramos acceder a sus datos) mediante el método del retardador o el método extendido. Aquellos alumnos que hayan realizado la práctica 4A en el laboratorio deben utilizar el método del retardador (para no repetir lo que hicieron en el laboratorio), mientras que los que realicen la práctica 4B deben escoger el método extendido (que es más completo).

## Cálculo 

In [ ]:
measures = [
    AnglesField(0, 0, 0, 0),  # 1
    AnglesField(0, 0, 90, 0),
    AnglesField(90, 0, 0, 0),
    AnglesField(90, 0, 90, 0),
    AnglesField(0, 0, 0, -45),  # 5
    AnglesField(0, 0, 45, 0),
    AnglesField(0, 45, 0, 0),
    AnglesField(45, 0, 0, 0),
    AnglesField(90, 0, 0, -45),
    AnglesField(90, 0, 45, 0),  # 10
]

from typing import List

# https://doi.org/10.1016/j.optlaseng.2021.106914
def extended_method(Is: List[np.array]) -> Jones_matrix:
    I1, I2, I3, I4, I5, I6, I7, I8, I9, I10 = Is
    J00 = np.sqrt(I1)
    J01 = np.sqrt(I2)
    J10 = np.sqrt(I3)
    J11 = np.sqrt(I4)
    delta_0 = 0
    delta_1 = np.arctan2(2 * I5 - I1 - I2, 2 * I6 - I1 - I2)
    delta_2 = np.arctan2(2 * I7 - I1 - I2, 2 * I8 - I1 - I2)
    delta_3 = delta_2 + np.arctan2(2 * I9 - I3 - I4, 2 * I10 - I3 - I4)

    components = [
        J00 * np.exp(1j * delta_0),
        J01 * np.exp(1j * delta_1),
        J10 * np.exp(1j * delta_2),
        J11 * np.exp(1j * delta_3),
    ]
    Jslm = Jones_matrix()
    Jslm.from_components(components)
    return Jslm


from functools import partial


def measure_intensity(angles_360: AnglesField, J: Jones_matrix):
    assert isinstance(angles_360, AnglesField)
    angles = angles_degree_to_rad(angles_360)
    Eout = field_transmission(angles, J)
    intensity, _ = parameters_intensity_phase(Eout)
    return intensity


def do_extended_measures(*, measures):
    intensities = [measure_intensity(measure, J=Jslm) for measure in measures]
    J = extended_method(intensities)
    return J


J_experiment = do_extended_measures(measures=measures)

## Comparación 

Compara el módulo y la fase global de los elementos de matriz obtenidos "experimentalmente" con los de la matriz Jslm original.

In [ ]:
# out=Jslm.draw(verbose=False);
from itertools import count
import matplotlib.pyplot as plt


def plot_components(J: Jones_matrix, title: str):
    components = J.parameters.components()
    x = np.arange(0, 256)
    fig, axs = plt.subplots(4, 2)
    fig.tight_layout()
    for index, c, (col_abs, col_phi) in zip(count(), components, axs):
        col_abs.plot(x, np.abs(c))
        col_abs.set_title(rf"$|J_{{ {index:0>2b} }}| $")
        col_phi.set_title(rf"$\phi(J_{{ {index:0>2b} }}) $")
        col_phi.plot(x, np.angle(c))
    fig.suptitle(title)

In [ ]:
plot_components(Jslm, "Loaded from data")

plot_components(J_experiment, "Calculated on simulation")

## Discusión

Discute el resultado obtenido.

El resultado es consistente, es cierto que existen ciertos errores numericos que son muy visibles en angulos cerca de $\pi/2$, ya que en ese angulo los errores explotan. En el paper original, se describe que para mejorar los errores numéricos se puede usar unos calculos de intensidades adicionales para corregir estos errores.